Author:

Thomas Ulrich


Surface waves and their dispersion are the main focus of this notebook. 
Using synthetic seismograms generated with Instaseis (http://instaseis.net/), we try to recover the dispersion curves of Love and Rayleigh waves of the PREM model.

In [ ]:
import instaseis
import obspy
from obspy.taup.taup_geo import calc_dist
import matplotlib.pyplot as plt
import numpy as np


def removeTopRightAxis(ax):
    # remove top right axis
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

We first generate synthetic seismograms with Instaseis by specifying moment tensor, source and receiver locations.

In [ ]:
db = instaseis.open_db("syngine://prem_i_2s")
# Source
lats = 89.91
lons = 74.4940
# receiver
latr = 42.6390
lonr = 74.4940

receiver = instaseis.Receiver(latitude=latr, longitude=lonr, network="AB", station="CED")
t0 = obspy.UTCDateTime(2011, 1, 2, 3, 4, 5)
source = instaseis.Source(
    latitude=lats,
    longitude=lons,
    depth_in_m=12000,
    m_rr=4.710000e24 / 1e7,
    m_tt=3.810000e22 / 1e7,
    m_pp=-4.740000e24 / 1e7,
    m_rt=3.990000e23 / 1e7,
    m_rp=-8.050000e23 / 1e7,
    m_tp=-1.230000e24 / 1e7,
    origin_time=t0,
)
st = db.get_seismograms(
    source=source, receiver=receiver, components=["Z", "R", "T"], kind="displacement"
)
st.plot()

## Group Velocity

If dispersion occurs, waves of different wavelengths travel at different speeds.  
In the following, we band-pass filter the seismograms around various central periods, ranging from 5 to 250s.

In [ ]:
myPeriods = [5.0, 7.5, 10, 12.5, 15, 20, 50, 100, 150, 200, 250]
nper = len(myPeriods)

figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 9), dpi=160, sharex=True, sharey=False)
for i in range(nper + 1):
    removeTopRightAxis(axarr[i])

iZRT = 2
axarr[0].plot(st[iZRT].times(reftime=t0), st[iZRT].data, label="no filter")
axarr[0].legend()
print("BP filter central frequency, time of envelope maximum")
pickedArrival = np.zeros((nper, 1))
for i, T in enumerate(myPeriods):
    st_temp = st.copy()
    st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
    axarr[i + 1].plot(
        st_temp[iZRT].times(reftime=t0),
        st_temp[iZRT].data,
        label="T=%.1fs" % T,
    )
    # Envelope of filtered data
    data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
    axarr[i + 1].plot(st_temp[iZRT].times(reftime=t0), data_envelope.data)
    idmax = np.argmax(data_envelope.data)
    print(T, st_temp[iZRT].times(reftime=t0)[idmax])
    pickedArrival[i] = st_temp[iZRT].times(reftime=t0)[idmax]
    axarr[i + 1].legend()


axarr[nper].set_xlabel("time (s)")
axarr[nper // 2].set_ylabel("displacement (m)")
plt.show()

To compute the group velocity, we compute the distance between source and receivers using obspy.

In [ ]:
dist_degree_sr = calc_dist(
    source_latitude_in_deg=lats,
    source_longitude_in_deg=lons,
    receiver_latitude_in_deg=latr,
    receiver_longitude_in_deg=lonr,
    radius_of_planet_in_km=6371,
    flattening_of_planet=0,
)
dist_km_sr = 6371 * dist_degree_sr * 2 * np.pi / 360.0
print(f"distance source-receivers: {dist_km_sr:.1f} km ({dist_degree_sr:.1f} deg)")

Now we plot the inferred dispersion curves and compare them with theoretical estimates.

In [ ]:
plt.plot(myPeriods, dist_km_sr / pickedArrival, "x", label="this notebook")
plt.xlabel("frequency (Hz)")
plt.ylabel("group velocity (km/s)")


# source: http://online.kitp.ucsb.edu/online/earth_m06/masters2/oh/08.html
PREM_groupVelRayleigh = """19.54918	3.352409
22.592722	3.5495863
29.88685	3.7826443
48.336567	3.9184444
70.88198	3.88144
108.34908	3.7869449
163.06877	3.6745753
207.68121	3.600516
245.99522	3.5769787
271.7795	3.611183
298.8952	3.7124586
"""
# source: http://online.kitp.ucsb.edu/online/earth_m06/masters2/oh/08.html
PREM_groupVelLove = """
19.49333	3.26055
23.80026	3.4672015
28.953934	3.7215185
37.50143	3.9565687
44.815434	4.1042824
60.388584	4.1978517
83.67851	4.264917
122.87113	4.283009
160.17804	4.28892
217.57329	4.2980137
264.88776	4.321354
299.29742	4.3386884
"""


# a = np.fromstring(PREM_groupVelRayleigh,sep='	')
a = np.fromstring(PREM_groupVelLove, sep="	")
PREM_groupVel = a.reshape((len(a) // 2, 2))
plt.plot(PREM_groupVel[:, 0], PREM_groupVel[:, 1], label="PREM (Gabi Laske)")
plt.legend()
plt.show()

## Phase velocity

The frequency-dependent phase of a seismogram can be obtained by taking the Fourrier transform of it. It can be written as follows: 

\begin{equation*}
\Phi(\omega) = \omega t - k(\omega) x + \Phi_i(\omega) + 2n\pi = \omega t - \omega x / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}

Where $\omega t - \omega x / c(\omega)$ is the phase due to the propagation in space and time,  
$\Phi_i(\omega)$ depends on the source and  
$2n\pi$ reflects the periodicity of the phase.  
To compute the phase velocity we need to track the same wave at two receivers.  
Assuming that the receivers are at distance $x_1$ and $x_2$ from the source, and the wave passes at time $t_1$ and $t_2$ at each receivers, then the phase of both receivers would be:  

\begin{equation*}
\Phi_1(\omega) =   \omega t_1 - \omega x_1 / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}
and  
\begin{equation*}
\Phi_2(\omega) = \omega t_2 - \omega x_2 / c(\omega)+ \Phi_i(\omega) + 2m\pi
\end{equation*}

Then c can be obtained from the difference $\Phi_{21}(\omega)$ between $\Phi_2(\omega)$ and $\Phi_1(\omega)$:
\begin{equation*}
c(\omega) = \omega (x_2-x_1)/(\omega (t_2-t_1)+ 2(m-n)\pi- \Phi_{21}(\omega))
\end{equation*}
the term m-n is found empirically ensuring that the phase velocity is reasonable for long periods.  
source: Stein and Wysession.

In [ ]:
receiver2 = instaseis.Receiver(latitude=latr - 6, longitude=lonr)
st2 = db.get_seismograms(
    source=source, receiver=receiver2, components=["Z", "R", "T"], kind="displacement"
)
st.plot()
st2.plot()

In [ ]:
myPeriods = [50, 100, 150, 200, 250]
nper = len(myPeriods)

figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 9), dpi=160, sharex=True, sharey=False)
for i in range(nper + 1):
    removeTopRightAxis(axarr[i])
iZRT = 2
axarr[0].plot(st[iZRT].times(reftime=t0), st[iZRT].data, label="no filter, r1")
axarr[0].plot(st2[iZRT].times(reftime=t0), st2[iZRT].data, label="no filter, r2")
axarr[0].legend()

pickedArrival = np.zeros((nper, 3))
pickedArrival[:, 0] = myPeriods
for i, T in enumerate(myPeriods):
    for k, sti in enumerate([st, st2]):
        st_temp = sti.copy()
        st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
        axarr[i + 1].plot(
            st_temp[iZRT].times(reftime=t0),
            st_temp[iZRT].data,
            label="T=%.1fs" % T,
        )
        # Envelope of filtered data
        data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
        axarr[i + 1].plot(st_temp[iZRT].times(reftime=t0), data_envelope.data)
        idmax = np.argmax(data_envelope.data)
        pickedArrival[i, k + 1] = st_temp[iZRT].times(reftime=t0)[idmax]
        axarr[i + 1].legend()
print(pickedArrival)

axarr[5].set_xlabel("time (s)")
axarr[5].set_ylabel("displacement (m)")
plt.show()